# Housing prices in Hyderabad, India

## Project Objective 🎯

The objective of this project is to develop a regression model to predict housing prices in Hyderabad, India. Using features such as the property's area, location, number of bedrooms, and available amenities, the model will aim to estimate the market value of a property as accurately as possible.

- This predictive model will be a valuable tool for:
- Home Buyers and Sellers: To obtain an objective price estimate for a property.
- Real Estate Agents: To assist with property valuation and client advisory.
- Investors: To identify potentially undervalued or overvalued properties in the market.

## 5. Evaluate the models

### 5.1 Loading the dataset

In [11]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, root_mean_squared_error
import joblib
import sys
sys.path.append('../../src/utils')

# Utilities
from regresion_metrics_column_definition import Metric


metrics = '../../datasets/processed/housing_prices/hyderabad_house_price_metrics.csv'
neuronal_network_metrics = '../../datasets/processed/housing_prices/hyderabad_house_price_neural_network_metrics.csv'
neuronal_network_xgboost = '../../datasets/processed/housing_prices/hyderabad_house_price_xgboost_metrics.csv'

best_model_without_pca = '../../datasets/processed/housing_prices/best_hyderabad_house_price_model.pkl'

training_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_features.parquet')
training_labels = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_labels.parquet')

test_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_features.parquet')
test_labels= pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_labels.parquet')

metrics_dataset = pd.read_csv(metrics)
neuronal_network_dataset = pd.read_csv(neuronal_network_metrics)
xgboost_dataset = pd.read_csv(neuronal_network_xgboost)

models_without_pca = metrics_dataset[~metrics_dataset['model_name'].str.contains('PCA')]
models_with_pca = metrics_dataset[metrics_dataset['model_name'].str.contains('PCA')]


### 5.2 Evaluate Models without PCA

In [12]:
# weight for rmse and r2
w_rmse = 1.0
w_r2 = 1.0

# Normalize and get the rmse cost
rmse = -models_without_pca[Metric.MEAN_TEST_NEG_RMSE.name]
delta_rmse = rmse.max() - rmse.min()
cost_rmse = (rmse - rmse.min()) / delta_rmse if delta_rmse > 0 else 0

# Normalize and get the r2 cost
r2 = models_without_pca[Metric.MEAN_TEST_R2.name]
delta_r2 = r2.max() - r2.min()
cost_r2 = (r2.max() - r2) / delta_r2 if delta_r2 > 0 else 0

# Getting the custom score
models_without_pca[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2)

# Finding the best model
best_model_idx = models_without_pca[Metric.SCORE.name].idxmin()
best_model_info = models_without_pca.loc[best_model_idx]


# --- Show Results ---
print("--- Best found Model without PCA ---")
print(f"Index: {best_model_idx}")
print(f"Model Name: {best_model_info['model_name']}")
print(f"Score: {best_model_info[Metric.SCORE.name]:.8f} (smaller is better)")
print("\nModel Metrics:")
print(f"  -> RMSE: {-best_model_info['mean_test_neg_rmse']:.4f}")
print(f"  -> R2 Score: {best_model_info['mean_test_r2']:.4f}")
print("\Hyperparameter:")
print(best_model_info['params'])

--- Best found Model without PCA ---
Index: 212
Model Name: L2
Score: 0.00000098 (smaller is better)

Model Metrics:
  -> RMSE: 0.2313
  -> R2 Score: 0.8660
\Hyperparameter:
{'regresion__alpha': np.float64(0.004520353656360241)}


<>:31: SyntaxWarning: invalid escape sequence '\H'
<>:31: SyntaxWarning: invalid escape sequence '\H'
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\2516717372.py:31: SyntaxWarning: invalid escape sequence '\H'
  print("\Hyperparameter:")
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\2516717372.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  models_without_pca[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2)


### 5.3 Evaluate Models without PCA

In [13]:
# weight for rmse and r2 and components
w_rmse = 1.0
w_r2 = 1.0
w_pca = 0.5 

# Normalize and get the rmse cost
rmse = -models_with_pca[Metric.MEAN_TEST_NEG_RMSE.name]
delta_rmse = rmse.max() - rmse.min()
cost_rmse = (rmse - rmse.min()) / delta_rmse if delta_rmse > 0 else 0

# Normalize and get the r2 cost
r2 = models_with_pca[Metric.MEAN_TEST_R2.name]
delta_r2 = r2.max() - r2.min()
cost_r2 = (r2.max() - r2) / delta_r2 if delta_r2 > 0 else 0

# Normalize and get the components cost
components = models_with_pca[Metric.PCA_COMPONENTS.name]
delta_components = components.max() - components.min()
cost_components = (components - components.min()) / delta_components if delta_components > 0 else 0

# Getting the custom score
models_with_pca[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2) + (w_pca * cost_components)

# Finding the best model
best_model_idx = models_with_pca[Metric.SCORE.name].idxmin()
best_model_info = models_with_pca.loc[best_model_idx]

# --- Show Results ---
print("--- Best found Model with PCA ---")
print(f"Index: {best_model_idx}")
print(f"Model Name: {best_model_info['model_name']}")
print(f"Score: {best_model_info[Metric.SCORE.name]:.8f} (smaller is better)")
print("\nModel Metrics:")
print(f"  -> RMSE: {-best_model_info['mean_test_neg_rmse']:.4f}")
print(f"  -> R2 Score: {best_model_info['mean_test_r2']:.4f}")
print("\Hyperparameter:")
print(best_model_info['params'])

--- Best found Model with PCA ---
Index: 1527
Model Name: PCA+L1
Score: 0.30556237 (smaller is better)

Model Metrics:
  -> RMSE: 0.2595
  -> R2 Score: 0.8325
\Hyperparameter:
{'pca__n_components': 35, 'regresion__alpha': np.float64(0.00014873521072935117)}


<>:36: SyntaxWarning: invalid escape sequence '\H'
<>:36: SyntaxWarning: invalid escape sequence '\H'
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\4142159469.py:36: SyntaxWarning: invalid escape sequence '\H'
  print("\Hyperparameter:")
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\4142159469.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  models_with_pca[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2) + (w_pca * cost_components)


### 5.4 Evaluate Neuronal Network

In [14]:
# weight for rmse and r2
w_rmse = 1.0
w_r2 = 1.0

# Normalize and get the rmse cost
rmse = -neuronal_network_dataset[Metric.MEAN_TEST_NEG_RMSE.name]
delta_rmse = rmse.max() - rmse.min()
cost_rmse = (rmse - rmse.min()) / delta_rmse if delta_rmse > 0 else 0

# Normalize and get the r2 cost
r2 = neuronal_network_dataset[Metric.MEAN_TEST_R2.name]
delta_r2 = r2.max() - r2.min()
cost_r2 = (r2.max() - r2) / delta_r2 if delta_r2 > 0 else 0

# Getting the custom score
neuronal_network_dataset[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2)

# Finding the best model
best_model_idx = neuronal_network_dataset[Metric.SCORE.name].idxmin()
best_model_info = neuronal_network_dataset.loc[best_model_idx]


# --- Show Results ---
print("--- Best found Neuronal Network ---")
print(f"Index: {best_model_idx}")
print(f"Model Name: {best_model_info['model_name']}")
print(f"Score: {best_model_info[Metric.SCORE.name]:.8f} (smaller is better)")
print("\nModel Metrics:")
print(f"  -> RMSE: {-best_model_info['mean_test_neg_rmse']:.4f}")
print(f"  -> R2 Score: {best_model_info['mean_test_r2']:.4f}")
print("\Hyperparameter:")
print(best_model_info['params'])

--- Best found Neuronal Network ---
Index: 2
Model Name: NN
Score: 0.00157367 (smaller is better)

Model Metrics:
  -> RMSE: 0.2571
  -> R2 Score: 0.8342
\Hyperparameter:
{'regresion__activation': 'relu', 'regresion__alpha': 0.0001, 'regresion__hidden_layer_sizes': (128, 64, 32), 'regresion__solver': 'adam'}


<>:31: SyntaxWarning: invalid escape sequence '\H'
<>:31: SyntaxWarning: invalid escape sequence '\H'
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\1687573477.py:31: SyntaxWarning: invalid escape sequence '\H'
  print("\Hyperparameter:")


### 5.5 Evaluate XGBoost

In [15]:
# weight for rmse and r2
w_rmse = 1.0
w_r2 = 1.0

# Normalize and get the rmse cost
rmse = -xgboost_dataset[Metric.MEAN_TEST_NEG_RMSE.name]
delta_rmse = rmse.max() - rmse.min()
cost_rmse = (rmse - rmse.min()) / delta_rmse if delta_rmse > 0 else 0

# Normalize and get the r2 cost
r2 = xgboost_dataset[Metric.MEAN_TEST_R2.name]
delta_r2 = r2.max() - r2.min()
cost_r2 = (r2.max() - r2) / delta_r2 if delta_r2 > 0 else 0

# Getting the custom score
xgboost_dataset[Metric.SCORE.name] = (w_rmse * cost_rmse) + (w_r2 * cost_r2)

# Finding the best model
best_model_idx = xgboost_dataset[Metric.SCORE.name].idxmin()
best_model_info = xgboost_dataset.loc[best_model_idx]


# --- Show Results ---
print("--- Best found XGBoost ---")
print(f"Index: {best_model_idx}")
print(f"Model Name: {best_model_info['model_name']}")
print(f"Score: {best_model_info[Metric.SCORE.name]:.8f} (smaller is better)")
print("\nModel Metrics:")
print(f"  -> RMSE: {-best_model_info['mean_test_neg_rmse']:.4f}")
print(f"  -> R2 Score: {best_model_info['mean_test_r2']:.4f}")
print("\Hyperparameter:")
print(best_model_info['params'])

--- Best found XGBoost ---
Index: 11
Model Name: XGB
Score: 0.00000000 (smaller is better)

Model Metrics:
  -> RMSE: 0.2321
  -> R2 Score: 0.8655
\Hyperparameter:
{'regresion__learning_rate': 0.1, 'regresion__max_depth': 3, 'regresion__n_estimators': 300}


<>:31: SyntaxWarning: invalid escape sequence '\H'
<>:31: SyntaxWarning: invalid escape sequence '\H'
C:\Users\luis.bolanoss\AppData\Local\Temp\ipykernel_23400\640513487.py:31: SyntaxWarning: invalid escape sequence '\H'
  print("\Hyperparameter:")


### 5.6 Generating the Best Model Artifact 

In [16]:
best_model = Ridge(alpha=0.004520353656360241)
best_model.fit(training_features, training_labels)

y_pred_test = best_model.predict(test_features)
rmse_test = root_mean_squared_error(test_labels, y_pred_test)
r2_test = r2_score(test_labels, y_pred_test)

print("\n--- Best Model Performance (L2) on the Test Set ---")
print("\nTest Model Metrics:")
print(f"RMSE: {rmse_test:.4f}")
print(f"R2 Score: {r2_test:.4f}")

joblib.dump(best_model, best_model_without_pca)


--- Best Model Performance (L2) on the Test Set ---

Test Model Metrics:
RMSE: 0.2069
R2 Score: 0.8894


['../../datasets/processed/housing_prices/best_hyderabad_house_price_model.pkl']